# I made this notebook to start in the middle because I don't want to keep regenerating the test data

In [5]:
# Read embedding model datasets that have already been created and stored
import readDataSets
test_dataset, training_dataset, val_dataset = readDataSets.loadDataSets()


In [6]:
# Load PDF documents from a directory
import loadReferenceDocuments
separate_pages, one_document = loadReferenceDocuments.loadReferenceDocuments("References/")

<built-in method count of list object at 0x1427aec00>


In [8]:
# Try to make a vector store
import splitAndVectorize
vectorStore = splitAndVectorize.createVectorstore(
    separate_pages,
    "separate_page_collection",
)
client = vectorStore.client
print(client.get_collections())


collections=[CollectionDescription(name='separate_page_collection')]


In [9]:
vectorStore = splitAndVectorize.createVectorstore(
    one_document,
    "One_Doc_Split_collection",
    chunk_size=500,
    chunk_overlap=100,

)
client = vectorStore.client
print(client.get_collections())

collections=[CollectionDescription(name='One_Doc_Split_collection')]
